In [ ]:
%reload_ext autoreload
%autoreload 2

# player

> Fetch player starts for individual games.

In [ ]:
#| default_exp player

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import typing
from collections import Counter
from math import sqrt

In [ ]:
#| export
def fetch_all_players():
    """Fetches all players in the rugbypy manifest file."""
    try:
        player_manifest_url = "https://raw.githubusercontent.com/seanyboi/rugbydata/main/data/v2/player/player_manifest.parquet"
        players = pd.read_parquet(player_manifest_url, engine="pyarrow")
        return players
    except Exception as e:
        print(
            f"No player manifest exists. Please raise an issue! - {e}"
        )

In [ ]:
#| exporti
def word2vec(word):

    # Count the number of characters in each word.
    count_characters = Counter(word)

    # Gets the set of characters and calculates the "length" of the vector.
    set_characters = set(count_characters)

    length = sqrt(sum(c*c for c in count_characters.values()))

    return count_characters, set_characters, length, word

In [ ]:
#| exporti
def cosine_similarity(vector1, vector2, ndigits):
    
    # Get the common characters between the two character sets
    common_characters = vector1[1].intersection(vector2[1])

    # Sum of the product of each intersection character.
    product_summation = sum(vector1[0][character] * vector2[0][character] for character in common_characters)

    # Gets the length of each vector from the word2vec output.
    length = vector1[2] * vector2[2]

    # Calculates cosine similarity and rounds the value to ndigits decimal places.
    if length == 0:
        # Set value to 0 if word is empty.
        similarity = 0
    else:
        similarity = round(product_summation/length, ndigits)

    return similarity

In [ ]:
#| export
def fetch_player(name: str # the name of the rugby player you wish to fetch the id for.
                ):
    """Uses a similarity tool to fetch a players id"""
    players = fetch_all_players()
    all_players = players["player_name"].to_list()
    # Initiate an empty list to store results.
    results_list = []

    # Apply word2vec function to each name and store them in a list.
    vector_list = [word2vec(str(i)) for i in all_players]
    vector_name = word2vec(str(name))
    # Two loops to compare each vector with another vector only once.
    for i in range(len(vector_list)):
        # Get first vector
        vector1 = vector_list[i]

        # Calculate cosine similarity
        similarity_score = cosine_similarity(vector1, vector_name, 3)

        # Append to results list if similarity score is between 1 and the threshold.
        # Note that scores of 1 can be ignored here if we want to exclude people with the same name.
        if 1 >= similarity_score >= 0.75:
            results_list.append([vector1[3], vector_name[3], similarity_score])

        else:
            pass
    # Convert list to dataframe.
    results_df = pd.DataFrame(results_list)
    if len(results_df) != 0:
        results_df.columns = ['player_name', 'comparison_name', 'similarity_score']
        results_df = results_df.sort_values(by="similarity_score", ascending=False).head(3)
        player_id = players.loc[players['player_name'].isin(results_df["player_name"].to_list())]
        return player_id
    else:
        # Can add error here if there's no results to return if desired.
        print(f"Apologies, we could not find a match for {name}. Please try again or raise an issue!")


In [ ]:
fetch_player("johnny sexton")

,player_id,player_name
1345,16004,Johnny Sexton
2156,291349,Ayden Johnstone
2795,149315,Anthony Watson


In [ ]:
#| export
def fetch_player_stats(player_id: str, # the player id of a particular player.
                       date: typing.Optional[str] = None # the date of a particular match you wish to fetch the player stats for.
                       ):
    """
    Fetches all player stats for a particular player or if a date is passed then just for a particular game.
    """
    try:
        if date:
            print(f"Fetching player stats for player_id: {player_id} on date: {date}...")
            player_url = f"https://raw.githubusercontent.com/seanyboi/rugbydata/main/data/v2/player/{player_id}.parquet"
            player_stats = pd.read_parquet(player_url, engine="pyarrow")
            player_stats = player_stats.query('game_date == @date')
            return player_stats
        else:
            print(f"Fetching all player stats for player_id: {player_id}...")
            player_url = f"https://raw.githubusercontent.com/seanyboi/rugbydata/main/data/v2/player/{player_id}.parquet"
            player_stats = pd.read_parquet(player_url, engine="pyarrow")
            return player_stats
    except Exception as e:
        print(
            f"No player stats for {player_id} because the player id does not exist. Please raise an issue! - {e}"
        )

In [ ]:
fetch_player_stats(player_id="16004")

Fetching all player stats for player_id: 16004...


,player_id,game_date,name,team,team_id,competition_id,competition,match_id,team_vs,team_vs_id,...,rucks_won,runs,tackles,total_free_kicks_conceded,total_lineouts,tries,try_assists,turnover_knock_on,turnovers_conceded,yellow_cards
0,16004,20220205,Johnny Sexton,Ireland,3,180659,Six Nations,595080,Wales,4,...,4.0,9.0,7.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0
1,16004,20220227,Johnny Sexton,Ireland,3,180659,Six Nations,595088,Italy,20,...,1.0,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,16004,20220312,Johnny Sexton,Ireland,3,180659,Six Nations,595091,England,1,...,4.0,12.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16004,20220319,Johnny Sexton,Ireland,3,180659,Six Nations,595093,Scotland,2,...,3.0,3.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,16004,20220610,Johnny Sexton,Leinster,25924,270557,United Rugby Championship,594483,Bulls,25953,...,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,16004,20221216,Johnny Sexton,Leinster,25924,271937,European Rugby Champions Cup,597405,Gloucester Rugby,25900,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
6,16004,20230204,Johnny Sexton,Ireland,3,180659,Six Nations,596205,Wales,4,...,3.0,8.0,7.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,16004,20230211,Johnny Sexton,Ireland,3,180659,Six Nations,596208,France,9,...,1.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,16004,20230312,Johnny Sexton,Ireland,3,180659,Six Nations,596216,Scotland,2,...,2.0,5.0,9.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,16004,20230318,Johnny Sexton,Ireland,3,180659,Six Nations,596219,England,1,...,6.0,9.0,5.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0


In [ ]:
fetch_player_stats(player_id="16004", date="20231014")

Fetching player stats for player_id: 16004 on date: 20231014...


,player_id,game_date,name,team,team_id,competition_id,competition,match_id,team_vs,team_vs_id,...,rucks_won,runs,tackles,total_free_kicks_conceded,total_lineouts,tries,try_assists,turnover_knock_on,turnovers_conceded,yellow_cards
14,16004,20231014,Johnny Sexton,Ireland,3,164205,Rugby World Cup,596195,New Zealand,8,...,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()